In [ ]:
!pip install rdkit-pypi
!pip install pandas==0.23.0
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

!pip install git+https://github.com/samoturk/mol2vec;

In [ ]:
import pandas as pd
# นำเข้าข้อมูล
mdf = pd.read_csv('Drug_FDA_2025.csv')

smiles = mdf['SMILES'].values



In [ ]:
# Dim 100ฝ300 ผ่านแล้ว
import numpy as np
import pandas as pd
from mol2vec.features import mol2alt_sentence, MolSentence, sentences2vec
from gensim.models import Word2Vec
from rdkit import Chem  # Import the Chem module

# Change to the 100-dimensional Mol2Vec model 
model = Word2Vec.load("mlchem/model_300dim.pkl") # 
mdf['mol'] = mdf['Smiles'].apply(lambda x: Chem.MolFromSmiles(x))

# Replace 'model.wv.vocab' with 'model.wv.key_to_index'
def sentences2vec_updated(sentences, model, unseen='UNK'):
    keys = set(model.wv.key_to_index.keys())
    vec = []
    for word in sentences:
        if word in keys:
            vec.append(model.wv[word])
        else:
            vec.append(model.wv[unseen])
    return np.array(vec)  # Convert the list to a numpy array

# Function to apply Mol2Vec to the entire DataFrame and save to CSV
def save_mol2vec_features(mdf, model, output_csv='mol2vec_features.csv'):
    features = []
    for idx, row in mdf.iterrows():
        # Convert SMILES to Mol object
        mol_sentence = mol2alt_sentence(row['mol'], radius=9) # Change Radius 0-9
        sentence_obj = MolSentence(mol_sentence)
        
        # Get the Mol2Vec feature vector for the sentence
        vector = sentences2vec_updated(sentence_obj, model, unseen='UNK')
        
        # No need to truncate or pad if you're using the 100-dimensional Mol2Vec model
        # Perform averaging across the word vectors to create a single vector
        vector = np.mean(vector, axis=0)

        # Flatten the vector into a 1D array
        features.append(vector.flatten())  # Flatten to a 1D array for each molecule
    
    # Convert list of feature vectors into a DataFrame
    feature_df = pd.DataFrame(features)
    
    # Save the DataFrame to a CSV file
    feature_df = pd.concat([feature_df, y_df], axis=1)
    feature_df.to_csv(output_csv, index=True)
    print(f"Mol2Vec features saved to {output_csv}")

# Assuming 'mdf' is your DataFrame and it has a 'mol' column with SMILES strings
# Apply the function to save features to CSV

save_mol2vec_features(mdf, model, output_csv='R9D300.csv')  # Using the 100-dimensional model
